# DenseNet121 PEFT on Cifar-10 with Cerebros

In [2]:
import sys
sys.path.insert(0, '../..')

In [41]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

## Fine-tuning some layers of DenseNet from scratch

This is done for comparison only, and may be skipped. This section provides no information on Cerebros' efficiency

Download DenseNet121 with Imagenet weights (1000 classes)

In [19]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

In [20]:
dnet.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_4[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

Make all layers untrainable except for the very last convolutional layer

In [21]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

Cifar-10 testing

In [22]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [23]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [24]:
def preprocess(x):
    x = tf.image.resize(x,size=(224,224),method='bicubic')
    x = tf.keras.applications.densenet.preprocess_input(x)
    return x

Modify the model

In [25]:
input_shape = (32,32,3)

In [26]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = dnet(prep)
dnet_mod = Model(inputs=input_layer, outputs=out)

In [27]:
dnet_mod.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-10 data and then evaluate (there is no hope this is gonna work ...)

In [28]:
dnet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 3842s 2s/step - loss: 5.9553 - top_1_categorical_accuracy: 0.1825


In [29]:
dnet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 648s 2s/step - loss: 4.8529 - top_1_categorical_accuracy: 0.2640


[4.8528923988342285, 0.2639999985694885]

## PEFT with Cerebros

Download Cifar-10 data

In [16]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Input and output shapes

In [29]:
input_shape   = (32,32,3)
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [10]

Subsampling a small balanced set of samples from the train dataset and train labels

In [18]:
def subsample_train(X_train, y_train, num_samples):
    #
    X_sub = []
    y_sub = []
    #
    assert 1 <= num_samples <= 5000
    #
    for cat in range(10):
        #
        ind, _ = np.where(y_train==cat)
        np.random.shuffle(ind)
        #
        X_cat = X_train[ind]
        X_cat = X_cat[:num_samples]
        y_cat = y_train[ind]
        y_cat = y_cat[:num_samples]
        #
        X_sub += [X_cat]
        y_sub += [y_cat]
    #
    X_sub = np.vstack(X_sub)
    y_sub = np.vstack(y_sub)
    #
    assert X_sub.shape[0] == y_sub.shape[0]
    #
    ind = np.arange(X_sub.shape[0])
    np.random.shuffle(ind)
    #
    return X_sub[ind], y_sub[ind]

We take only 200 samples in each category (out of 5000)

In [19]:
num_samples = 200
X_sub, y_sub = subsample_train(X_train, y_train, num_samples)

Preparing tensors for the training set and labels

In [20]:
training_x   = [tf.constant(X_sub)]
y_train_cat  = to_categorical(y_sub, 10)
train_labels = [tf.constant(y_train_cat)]

Donwloading DenseNet121

In [21]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

Preprocessing images for DenseNet

In [22]:
def preprocess(x):
    x = tf.image.resize(x,size=(224,224),method='bicubic')
    x = tf.keras.applications.densenet.preprocess_input(x)
    return x

Make only the last convolutional layer trainable

In [23]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

Preparing the base model for Cerebros search

In [24]:
dnet_io = Model(inputs=dnet.layers[0].input,
                outputs=dnet.layers[-2].output)

In [25]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = Flatten()(dnet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [30]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 5  # [1, 100]
batch_size = 20
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [31]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar10_densenet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [32]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_0_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_1_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_0_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_1_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'Ne

INFO:tensorflow:Assets written to: 2023_11_01_11_24_cerebros_auto_ml_test_cifar10_densenet_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.775338                    0.725625  0.517063   
1  0.204626                    0.946250  0.488564   
2  0.098013                    0.981875  0.519281   
3  0.064410                    0.990625  0.502375   
4  0.039212                    0.992500  0.546139   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                          0.8300             0                0   
1                          0.8525             0                0   
2                          0.8350             0                0   
3                          0.8400             0                0   
4                          0.8375             0                0   

                                          model_name  
0  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
1  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
2  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
3  2023_11_01_11_24_cerebr

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_/Softmax:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
______________________________________________________________________

...

INFO:tensorflow:Assets written to: 2023_11_01_11_24_cerebros_auto_ml_test_cifar10_densenet_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.698838                    0.782500  0.643460   
1  0.119176                    0.980000  0.517664   
2  0.048075                    0.998750  0.515128   
3  0.026445                    0.999375  0.507785   
4  0.016099                    1.000000  0.524440   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                          0.8000             0                0   
1                          0.8475             0                0   
2                          0.8350             0                0   
3                          0.8475             0                0   
4                          0.8400             0                0   

                                          model_name  
0  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
1  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
2  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
3  2023_11_01_11_24_cerebr

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000003_tr_0_FinalDenseUnit_0000000000000003_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_2')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_1_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_

INFO:tensorflow:Assets written to: 2023_11_01_11_24_cerebros_auto_ml_test_cifar10_densenet_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.650667                    0.801000  0.625180   
1  0.087494                    0.989375  0.496201   
2  0.033108                    0.999375  0.480476   
3  0.022173                    0.999375  0.489006   
4  0.014275                    1.000000  0.516827   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                          0.7875             0                1   
1                          0.8400             0                1   
2                          0.8350             0                1   
3                          0.8450             0                1   
4                          0.8300             0                1   

                                          model_name  
0  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
1  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
2  2023_11_01_11_24_cerebros_auto_ml_test_cifar10...  
3  2023_11_01_11_24_cerebr

In [33]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.8525000214576721
top-1 categorical accuracy


Evaluating the best model found

In [40]:
best_model_found = cerebros_automl.get_best_model()

In [35]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [36]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 model_2 (Functional)        (None, 1024)                 7037504   ['NeuralNetworkFuture_00000000
                                          

In [37]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 10)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 1009s 3s/step - loss: 0.4531 - eval_top_1_categorical_accuracy: 0.8534 - eval_top_5_categorical_accuracy: 0.9925


[0.4530925154685974, 0.8533999919891357, 0.9925000071525574]

Now train the best model on the entire train dataset

In [58]:
best_model_found = cerebros_automl.get_best_model()

In [59]:
y_train_cat  = to_categorical(y_train, 10)
#
optimizer = Adam(learning_rate=0.0005)
#
loss = tf.keras.losses.CategoricalCrossentropy()
#
metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]
#
best_model_found.compile(optimizer=optimizer, 
                         loss=loss, 
                         metrics=metrics, 
                         )

In [ ]:
%%time
best_model_found.fit(X_train,
                     y_train_cat,
                     validation_split=0.35,
                     epochs=10,
                    )

Epoch 1/10
1016/1016 [==============================] - 2909s 3s/step - loss: 0.3444 - eval_top_1_categorical_accuracy: 0.8865 - eval_top_5_categorical_accuracy: 0.9956 - val_loss: 0.2755 - val_eval_top_1_categorical_accuracy: 0.9051 - val_eval_top_5_categorical_accuracy: 0.9970
Epoch 2/10
1016/1016 [==============================] - 2697s 3s/step - loss: 0.2566 - eval_top_1_categorical_accuracy: 0.9130 - eval_top_5_categorical_accuracy: 0.9978 - val_loss: 0.2611 - val_eval_top_1_categorical_accuracy: 0.9097 - val_eval_top_5_categorical_accuracy: 0.9971
Epoch 3/10
1016/1016 [==============================] - 2704s 3s/step - loss: 0.2230 - eval_top_1_categorical_accuracy: 0.9219 - eval_top_5_categorical_accuracy: 0.9985 - val_loss: 0.2708 - val_eval_top_1_categorical_accuracy: 0.9079 - val_eval_top_5_categorical_accuracy: 0.9972
Epoch 4/10
1016/1016 [==============================] - 2729s 3s/step - loss: 0.1935 - eval_top_1_categorical_accuracy: 0.9333 - eval_top_5_categorical_accuracy

Evaluating again

In [ ]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
best_model_found.evaluate(X_test, y_test_cat)